## 作業

請閱讀 Keras 官方範例 [mnist_cnn.py](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)  

並回答下列問題。僅有 70 行程式碼，請確保每一行的程式碼你都能夠理解目的

1. 是否有對資料做標準化 (normalization)? 如果有，在哪幾行?
   ==> <font color='red'>Line 36 ~ 39</font>
2. 使用的優化器 Optimizer 為何?
   ==> <font color='red'>keras.optimizers.Adadelta (Line 61)</font>
3. 模型總共疊了幾層卷積層?
   ==> <font color='red'>二層 (Line 49, Line 52)</font>
4. 模型的參數量是多少? 
   ==> <font color='red'>總數量 : 320+18,496+1,179,776+1,290=1,199,882</font><br>
   <font color='red'>- 卷積層(1) : (3*3*1+1)*32=320</font><br>
   <font color='red'>- 卷積層(2) : (3*3*32+1)*64=18,496</font><br>
   <font color='red'>- 全連接層(1) : (9,216+1)*128=1,179,776</font><br>
   <font color='red'>- 全連接層(2) : (128+1)*10=1,290</font><br>
    
   <p style='background-color:#FDFED1'>卷積層 (Conv2D) 參數量計算 = (n * m * l + 1) * k <br>
    ==> l : 輸入特徵數量 / k : 輸出特徵數量 / n * m : 過濾器大小
   <br><br>
   池化層 (MaxPooling2D) 參數量計算 = 0 (沒有可以在池中學習的參數, 這一層只是用來減少圖像尺寸大小)
   <br><br>
   平坦層 (Flatten) 參數量計算 = 0 (此層功能為將矩陣打平成一維的陣列)
   <br><br>
   Dropout 層參數量計算 = 0 (此層功能為隨機捨棄輸入tensor 的某些元素, 防止 overfit)
   <br><br>
   全連接層 (Dense) 參數數量計算 = (n + 1) * m<br>
    ==> n : 輸入 / m : 輸出</p>

In [1]:
"""
Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
"""

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 208us/step - loss: 0.2591 - acc: 0.9203 - val_loss: 0.0614 - val_acc: 0.9801
Epoch 2/12
60000/60000 [==============================] - 7s 110us/step - loss: 0.0840 - acc: 0.9750 - val_loss: 0.0399 - val_acc: 0.9866
Epoch 3/12
60000/60000 [==============================] - 6s 106us/step - loss: 0.0632 - acc: 0.9807 - val_loss: 0.0336 - val_acc: 0.9890
Epoch 4/12
60000/60000 [==============================] - 6s 106us/step - loss: 0.0517 - acc: 0.9841 - val_loss: 0.0320 - val_acc: 0.9897
Epoch 5/12
60000/60000 [==============================] - 6s 106us/step - loss: 0.0467 - acc: 0.9857 - val_loss: 0.0301 - val_acc: 0.9899
Epoch 6/12
60000/60000 [==============================] - 6s 107us/step - loss: 0.0409 - acc: 0.9874 - val_loss: 0.0297 - val_acc: 0.9901
Epoch 7/12
60000/60000 [========================

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________